# 보안2 과정 함수화

1. 이미지 매칭
 - 평균 해시 매칭(Average Hash Matching)

2. 특징 디스크립터 검출기와 올바른 매칭점 찾기
 - 디스크립터 검출기: ORB (Oriented and Rotated BRIEF)
 - 매칭점 찾기 평가 지표: knnMatch()


#### 라이브러리

In [1]:

# Data handling
import numpy as np


# OpenCV, OCR
import cv2
import pytesseract
from skimage.transform import resize


# Youtube video
import pafy
import cv2


# Visualizatipn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='malgun gothic')
plt.rc('axes', unicode_minus=False)

# time
from time import time

#### 차량 진입 사진 로드

In [3]:
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype) 
        img = cv2.imdecode(n, flags) 
        return img 
    except Exception as e: 
        print(e) 
        return None

In [5]:
greencar01 = imread('./그린카진입01.jpg')
greencar02 = imread('./그린카진입02.jpg')

# 1. 이미지 매칭
 - 평균 해시 매칭(Average Hash Matching)

In [19]:
greencar01_hash = img2hash(greencar01)
greencar02_hash = img2hash(greencar02)

In [21]:
# 이미지를 16x16 크기의 평균 해쉬로 변환
def img2hash(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#    gray = cv2.resize(gray, (600, 300)) # 이미지 croped & resize된 사진을 받음
    avg = gray.mean()
    bi = 1 * (gray > avg)
    return bi

# 해밍거리 측정 함수
def hamming_distance(a, b):
    a = a.reshape(1,-1)
    b = b.reshape(1,-1)
    # 같은 자리의 값이 서로 다른 것들의 합
    distance = (a !=b).sum()
    return distance

def image_match(image, stored_hash):
    # 들어온 차량의 이미지
    income_car_image = image    
    # 차 영상의 해쉬 구하기
    query_hash = img2hash(income_car_image)
    
    # 해당 차량의 저장된 해쉬 불러오기
    stored_hash = stored_hash
    
    # 해밍 거리 산출
    flag = 0
    hamming_dst = hamming_distance(query_hash, stored_hash)
    if hamming_dst/256 < 200: flag=1; print('이미지매칭: 동일 차량입니다.')
    else: print('동일하지 않은 차량입니다.')
        
    return income_car_image, flag

In [23]:
print(image_match(greencar01, greencar01_hash)[1])
print(image_match(greencar01, greencar02_hash)[1])

이미지매칭: 동일 차량입니다.
1
이미지매칭: 동일 차량입니다.
1


# 2. ORB(Oriented and Rotated BRIEF)와 knnMatch
 knnMatch() 함수는 디스크립터당 k개의 최근접 이웃 매칭점을 가까운 순서대로 반환합니다. k개의 최근접 이웃 중 거리가 가까운 것은 좋은 매칭점이고, 거리가 먼 것은 좋지 않은 매칭점일 가능성이 높습니다. 최근접 이웃 중 거리가 가까운 것 위주로 골라내면 좋은 매칭점을 찾아낼 수 있습니다.

In [26]:
# 서술자 추출 함수
def get_desc(img):
    img = img
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kp, desc = detector.detectAndCompute(gray, None)
    return kp, desc

In [28]:
# ORB로 서술자 추출 
detector = cv2.ORB_create()

# BF-Hamming 생성
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)

def ORB_knnMatch(income_car_image, flag, stored_kp, stored_desc):
    # 각 영상에 대해 키 포인트와 서술자 추출 
    compare_kp, compare_desc = get_desc(income_car_image)
    # knnMatch, k=2
    matches = matcher.knnMatch(compare_desc, stored_desc, 2)

    # 첫번재 이웃의 거리가 두 번째 이웃 거리의 75% 이내인 것만 추출---⑤
    ratio = 0.75
    good_matches = [first for first,second in matches \
                        if first.distance < second.distance * ratio]
    if len(good_matches) < 70: print('특징점 비교: 동일 차량입니다.')
    else: print('동일하지 않은 차량입니다')
    #print('matches:%d/%d' %(len(good_matches),len(matches)))

    # 좋은 매칭만 그리기
    #res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, \
     #                   flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
    # 결과 출력
    #plt.figure(figsize=(15, 10))
    #plt.imshow(res)
    #plt.show()

In [30]:
greencar01_kp, greencar01_desc = get_desc(greencar01)
greencar02_kp, greencar02_desc = get_desc(greencar02)

In [32]:
# 이미지 매칭
income_car_image, flag = image_match(greencar01, greencar01_hash)

이미지매칭: 동일 차량입니다.


In [38]:
# 특징점 매칭
print(ORB_knnMatch(income_car_image, flag, greencar01_kp, greencar01_desc))
print(ORB_knnMatch(income_car_image, flag, greencar02_kp, greencar02_desc))



동일하지 않은 차량입니다
None
동일하지 않은 차량입니다
None


# 알고리즘 및 자료구조 사용 코드

In [40]:
import heapq  # 우선순위 큐를 위한 라이브러리

# 자료구조 추가: 우선순위 큐를 활용해 가장 유사한 이미지를 찾는 함수
def find_most_similar_image(query_hash, stored_hashes):
    """
    입력된 이미지의 해싱 값과 저장된 여러 이미지 해싱 값 중 
    가장 유사한 이미지(해밍 거리 기준)를 찾는다.
    
    query_hash: 비교하려는 입력 이미지의 해싱 값
    stored_hashes: 저장된 이미지들의 해싱 값 리스트
    """
    # 우선순위 큐 생성
    priority_queue = []  # (해밍 거리, 해싱 값) 형태로 저장

    for idx, stored_hash in enumerate(stored_hashes):
        hamming_dst = hamming_distance(query_hash, stored_hash)
        heapq.heappush(priority_queue, (hamming_dst, idx))  # 거리와 인덱스를 저장

    # 가장 유사한 이미지 추출
    best_match = heapq.heappop(priority_queue)  # 최솟값(가장 유사한 값) 추출
    print(f"가장 유사한 이미지: 저장된 해싱 {best_match[1]}번 (해밍 거리: {best_match[0]})")
    return best_match  # (해밍 거리, 인덱스)

# 기존 코드와 통합
# 저장된 해시 데이터 준비
stored_hashes = [greencar01_hash, greencar02_hash]

# 입력 이미지의 해시 계산
query_hash = img2hash(greencar01)

# 가장 유사한 이미지를 찾기
best_match = find_most_similar_image(query_hash, stored_hashes)

# 기존 이미지 매칭과 연결
if best_match[0] / 256 < 200:
    print("해싱 기반: 동일 차량입니다.")
else:
    print("해싱 기반: 동일하지 않은 차량입니다.")


가장 유사한 이미지: 저장된 해싱 0번 (해밍 거리: 0)
해싱 기반: 동일 차량입니다.


## 1. 우선순위 큐(Priority Queue)
#### 핵심 설명: 우선순위 큐는 데이터를 우선순위에 따라 정렬된 상태로 관리하는 자료구조입니다. 이 코드에서는 이미지의 해밍 거리를 기준으로 우선순위 큐를 사용하여 가장 유사한 이미지를 빠르게 찾습니다.
#### 알아두어야 할 점: heapq 라이브러리를 사용해 우선순위 큐를 구현합니다. heapq.heappush(priority_queue, (hamming_dst, idx))는 해밍 거리와 해당 이미지의 인덱스를 함께 큐에 삽입합니다. heapq.heappop(priority_queue)는 가장 작은 해밍 거리 값을 가진 항목을 추출합니다.
## 2. 해밍 거리(Hamming Distance)
#### 핵심 설명: 해밍 거리는 두 개의 이진 문자열이 얼마나 다른지 나타내는 지표입니다. 두 문자열에서 다른 비트가 몇 개인지를 계산하여 거리를 구합니다. 여기서는 이미지 해시 값 간의 유사도를 측정하는 데 사용됩니다.
#### 알아두어야 할 점: 이미지의 해시 값은 각 이미지의 특징을 숫자로 압축한 값입니다. 해밍 거리가 낮을수록 두 이미지가 더 유사하다고 판단됩니다. hamming_distance(query_hash, stored_hash) 함수는 두 해시 값 간의 해밍 거리를 계산합니다.
## 3. find_most_similar_image 함수
#### 핵심 설명: 이 함수는 입력 이미지의 해싱 값과 저장된 이미지들의 해싱 값을 비교하여, 가장 유사한 이미지를 찾습니다. 우선순위 큐를 사용하여 해밍 거리 순으로 정렬하고, 가장 작은 해밍 거리를 가진 이미지를 선택합니다.
#### 알아두어야 할 점: 함수는 query_hash와 stored_hashes를 입력받고, hamming_distance를 사용해 각 해시 간의 거리를 계산합니다. 이후 우선순위 큐에 (해밍 거리, 이미지 인덱스) 형태로 넣고, 가장 작은 거리를 가진 값을 추출하여 가장 유사한 이미지를 반환합니다.
## 4. 결과 출력 및 비교
#### 핵심 설명: 가장 유사한 이미지가 결정되면, 그 결과를 출력하고 해밍 거리 기준으로 이미지가 동일한지 아닌지를 판단합니다.
#### 알아두어야 할 점:
best_match[0] / 256 < 200 부분은 해밍 거리 값에 따라 "동일 차량"인지 "동일하지 않은 차량"인지를 판단하는 조건입니다.
해밍 거리가 200보다 작은 경우 "동일 차량"으로 간주하고, 그 이상일 경우 "동일하지 않은 차량"으로 판단합니다.
해밍 거리 값이 256 이하로 설정되어 있어, 최대 해밍 거리 값은 256입니다. 이 값은 해시 값이 8비트씩 계산되었을 때의 최대 차이를 의미합니다.
## 5. stored_hashes와 query_hash
#### 핵심 설명: stored_hashes는 미리 저장된 이미지들의 해시 값 리스트이며, query_hash는 비교하려는 입력 이미지의 해시 값입니다. 이 두 값의 유사도를 비교하여 가장 유사한 이미지를 찾습니다.
#### 알아두어야 할 점: stored_hashes는 여러 이미지를 담고 있으며, 비교하려는 이미지인 query_hash와 하나씩 비교하여 해밍 거리 계산 후, 가장 유사한 이미지를 찾습니다.
## 6. heapq 라이브러리 사용
#### 핵심 설명: heapq는 우선순위 큐를 구현하는 파이썬 내장 라이브러리입니다. heappush는 큐에 항목을 추가하고, heappop은 큐에서 최솟값을 반환합니다.
#### 알아두어야 할 점: heapq는 우선순위가 낮은(작은 값) 항목을 먼저 반환하므로, hamming_distance가 가장 작은 값이 가장 유사한 이미지가 됩니다. 이 방식은 효율적이고 빠른 매칭을 가능하게 합니다.
## 7. 출력
#### 핵심 설명: print(f"가장 유사한 이미지: 저장된 해싱 {best_match[1]}번 (해밍 거리: {best_match[0]})")를 통해 가장 유사한 이미지의 번호와 해밍 거리를 출력합니다.
#### 알아두어야 할 점: 이 출력은 best_match 튜플에서 0번 인덱스(해밍 거리)와 1번 인덱스(이미지 인덱스)를 참조하여 결과를 출력합니다.
## 8. 결과 해석
#### 핵심 설명: 해싱 값 기반으로 두 이미지가 동일한지 판단하는 기준을 설정할 때, 해밍 거리의 값이 작을수록 두 이미지가 유사하다고 판단합니다. 이 값을 기준으로 차량이 동일한지 여부를 판단할 수 있습니다.
#### 알아두어야 할 점: 해밍 거리 기준이 200을 기준으로 삼은 이유는 이미지 해싱에서 보통 작은 차이가 있을 때 유사한 이미지로 판단하기 위해 설정된 값입니다.
## 결론:
**우선순위 큐(Priority Queue)**를 사용해 효율적으로 가장 유사한 이미지를 찾고, 해밍 거리로 유사도를 측정하여 동일한 차량인지 판단하는 코드입니다.
heapq 라이브러리로 큐를 관리하여, 가장 유사한 이미지를 빠르게 찾을 수 있습니다.
해밍 거리를 기준으로 유사성을 판단하고, 유사도가 특정 기준을 넘으면 "동일 차량"으로 판단하는 구조입니다.